In [1]:
from models.BootstrapDY import estimate_var, make_dynamics, make_var_mbb, make_boot_dynamics, rolling_window, static, DieboldYilmaz2012
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('dy2012.csv', parse_dates=['Index'], index_col=0)
#data = data.loc['2007-1-1':'2008-1-1']
YY = data.values

In [3]:
nlags = 4
pp = nlags
Horizon = 10
TT = YY.shape[0] - nlags
forecast_horizon = Horizon

In [100]:
from statsmodels.tsa.api import VAR
model = VAR(YY)
results = model.fit(nlags)

In [5]:
statsA = results.coefs
for i in range(4):
	statsA[i] = statsA[i].T
statsA = statsA.reshape(16,4)
statsA = np.vstack((results.intercept.reshape((1,4)), statsA))

statsU = results.resid
stats_sigmacomp = results.sigma_u
statscovUU = (statsU.T @ statsU) / TT

statsPhi = results.ma_rep(9)
for I in range(10):
	statsPhi[i] = statsPhi[i].T


In [6]:
numObs, KK = YY.shape
fTT = numObs - nlags

fyy = YY[nlags:, :]
fxx = np.zeros((fTT, 1 + nlags*KK))
fxx[:, 0] = 1
for t in range(fTT):
    for p in range(nlags):
        fxx[t, 1+p*KK:1+KK+p*KK] = YY[t+nlags-p-1, :]
A_est, U_est, covUU_est = estimate_var(fyy, fxx)

# 进行系数的比较(使用sovle版)

In [18]:
np.fabs(A_est - statsA) #A有时差距较小有时差距较大

array([[9.21263066e-13, 1.32560629e-13, 7.62057084e-13, 7.90478794e-13],
       [4.55191440e-15, 9.23898864e-02, 1.91220139e-02, 4.26735851e-02],
       [9.23898864e-02, 5.35682609e-15, 4.56085347e-02, 2.36257513e-02],
       [1.91220139e-02, 4.56085347e-02, 4.74620343e-15, 2.31236051e-02],
       [4.26735851e-02, 2.36257513e-02, 2.31236051e-02, 2.88935542e-14],
       [4.88498131e-14, 2.27184873e-02, 2.48718457e-03, 3.18704010e-02],
       [2.27184873e-02, 3.09474668e-14, 3.80694633e-02, 3.46951451e-02],
       [2.48718457e-03, 3.80694633e-02, 1.28785871e-14, 3.35262082e-02],
       [3.18704010e-02, 3.46951451e-02, 3.35262082e-02, 4.02455846e-14],
       [4.62685446e-14, 5.27943399e-03, 3.40257466e-02, 1.32861261e-02],
       [5.27943399e-03, 1.76525461e-14, 4.77290233e-02, 2.59570507e-02],
       [3.40257466e-02, 4.77290233e-02, 2.60902411e-15, 3.35859918e-02],
       [1.32861261e-02, 2.59570507e-02, 3.35859918e-02, 3.13360449e-14],
       [2.21767049e-14, 5.94488533e-02, 2.72907220e

In [19]:
np.fabs(U_est - statsU) #扰动项差距较小

array([[1.33226763e-13, 1.65201186e-13, 2.48689958e-14, 1.65201186e-13],
       [3.90798505e-14, 1.06581410e-13, 9.05941988e-14, 3.01980663e-14],
       [3.55271368e-15, 1.61648472e-13, 9.59232693e-14, 3.37507799e-14],
       ...,
       [1.06581410e-14, 2.66453526e-14, 1.17239551e-13, 2.84217094e-14],
       [2.84217094e-14, 7.28306304e-14, 2.66453526e-14, 5.32907052e-15],
       [1.42108547e-14, 6.39488462e-14, 3.55271368e-14, 8.17124146e-14]])

In [22]:
np.fabs(covUU_est - statscovUU) # 如果按公式计算协方差较小

array([[1.11022302e-16, 5.55111512e-17, 1.38777878e-17, 8.32667268e-17],
       [5.55111512e-17, 2.22044605e-16, 5.55111512e-17, 5.55111512e-17],
       [1.38777878e-17, 5.55111512e-17, 2.22044605e-16, 2.77555756e-17],
       [8.32667268e-17, 5.55111512e-17, 2.77555756e-17, 0.00000000e+00]])

In [23]:
np.fabs(covUU_est - stats_sigmacomp) #如果直接拿到他给的协方差那差距较大

array([[0.0042642 , 0.00122055, 0.00046343, 0.00076953],
       [0.00122055, 0.00545408, 0.00041039, 0.00116751],
       [0.00046343, 0.00041039, 0.00876819, 0.00039462],
       [0.00076953, 0.00116751, 0.00039462, 0.00443586]])

In [24]:
np.fabs(statscovUU - stats_sigmacomp) #dy的程序是使用的sigmacomp

array([[0.0042642 , 0.00122055, 0.00046343, 0.00076953],
       [0.00122055, 0.00545408, 0.00041039, 0.00116751],
       [0.00046343, 0.00041039, 0.00876819, 0.00039462],
       [0.00076953, 0.00116751, 0.00039462, 0.00443586]])

$\frac{1}{T - Kp - 1} Y^\prime (I_T - Z (Z^\prime Z)^{-1} Z^\prime) Y$

In [32]:
results.sigma_u - ((statsU.T @ statsU) / (TT - KK*nlags -1)) #statsmodel这里进行了自由度调整

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [33]:
results.sigma_u - ((U_est.T @ U_est) / (TT - KK*nlags -1))

array([[-1.11022302e-16, -5.55111512e-17,  1.38777878e-17,
        -5.55111512e-17],
       [-5.55111512e-17, -2.22044605e-16, -4.16333634e-17,
         5.55111512e-17],
       [ 1.38777878e-17, -4.16333634e-17,  0.00000000e+00,
        -4.16333634e-17],
       [-5.55111512e-17,  5.55111512e-17, -4.16333634e-17,
         1.11022302e-16]])

# 进行系数的比较使用statsmodel的版本

In [7]:
np.fabs(A_est - statsA) #A有时差距较小有时差距较大

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
np.fabs(U_est - statsU) #扰动项差距较小

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [9]:
np.fabs(covUU_est - stats_sigmacomp) #如果直接拿到他给的协方差那差距较大

array([[0.00000000e+00, 0.00000000e+00, 1.38777878e-17, 5.55111512e-17],
       [0.00000000e+00, 2.22044605e-16, 2.77555756e-17, 5.55111512e-17],
       [1.38777878e-17, 2.77555756e-17, 2.22044605e-16, 1.38777878e-17],
       [5.55111512e-17, 5.55111512e-17, 1.38777878e-17, 1.11022302e-16]])

In [10]:
results.sigma_u - ((U_est.T @ U_est) / (TT - 17))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# 直接利用A和U进行测试

In [11]:
make_dynamics(A_est, covUU_est, 4, 10) # 计算结果仍然有差距

array([[0.8772493 , 0.08150053, 0.00489427, 0.03635589],
       [0.09122472, 0.82270893, 0.03492635, 0.05114001],
       [0.00331561, 0.02960581, 0.95160661, 0.01547196],
       [0.05509739, 0.0769218 , 0.02148512, 0.84649569]])

# 继续测试Phi的计算

In [137]:
AA = A_est

NN, KK = AA.shape                                       # AA这里是行观点系数
#coefs = AA[1:, :].reshape((pp, KK, KK))                # 1.提取非截距项部分 2.为了计算方便,将系数矩阵设为3维

coefs = np.zeros((pp, KK, KK))
for i in range(pp):
    coefs[i, :, :] = AA[1+i*KK:1+(i+1)*KK, :]

Phis = np.zeros((Horizon, KK, KK))
Phis[0] = np.eye(KK)

# recursively compute Phi matrices
for i in range(1, Horizon):
    for j in range(1, i+1):
        if j > pp:
            break

        Phis[i] += coefs[j-1] @ Phis[i-j]  # 注意这里Phis的下标没有位移(从Phi0开始), 但是AA的下标有位移

In [138]:
statsPhi = results.ma_rep(9)
for i in range(10):
	statsPhi[i] = statsPhi[i].T


In [139]:
np.fabs(Phis - statsPhi).sum()

0.0

# Phi也没有问题

In [174]:
AA = A_est

NN, KK = AA.shape                                       # AA这里是行观点系数
#coefs = AA[1:, :].reshape((pp, KK, KK))                # 1.提取非截距项部分 2.为了计算方便,将系数矩阵设为3维

coefs = np.zeros((pp, KK, KK))
for i in range(pp):
    coefs[i, :, :] = AA[1+i*KK:1+(i+1)*KK, :]

Phis = np.zeros((Horizon, KK, KK))
Phis[0] = np.eye(KK)

# recursively compute Phi matrices
for i in range(1, Horizon):
    for j in range(1, i+1):
        if j > pp:
            break

        Phis[i] += coefs[j-1] @ Phis[i-j]  # 注意这里Phis的下标没有位移(从Phi0开始), 但是AA的下标有位移

#================================================================================
CovUU = covUU_est
deviations = np.sqrt(np.diag(CovUU))# 这个没有问题

# for i in range(10):
# 	Phis[i] = Phis[i].T
#--------------
numerators = np.zeros((Horizon, KK, KK))
denominator = np.zeros((Horizon, KK, KK))
for h in range(Horizon):
    numerators[h] = Phis[h].T @ CovUU
    denominator[h] = numerators[h] @ Phis[h]
    numerators[h] = np.square(numerators[h]/ deviations)

DD = np.zeros((KK, KK))
for i in range(KK):
    for j in range(KK):
        DD[i, j] = np.sum(numerators[:, i, j])/np.sum(denominator[:, i, i])
#---------------


DD_tilde = DD/DD.sum(axis=1).reshape(KK,-1)*100

In [175]:
sigma_u = np.asarray(results.sigma_u)
sd_u = np.sqrt(np.diag(sigma_u))


#-----------------
fevd = results.fevd(forecast_horizon, sigma_u / sd_u)
fe = fevd.decomp[:, -1, :]
#-----------------


fevd = (fe / fe.sum(1)[:, None] * 100)

In [176]:
fevd

array([[88.75700164,  7.29118463,  0.34532792,  3.6064858 ],
       [10.21354465, 81.44571151,  2.72697368,  5.61377016],
       [ 0.46811796,  3.69595287, 93.69418926,  2.14173991],
       [ 5.69157879,  7.02601676,  1.54775915, 85.7346453 ]])

In [177]:
np.fabs(DD_tilde - fevd)

array([[1.42108547e-14, 8.88178420e-16, 3.33066907e-16, 3.10862447e-15],
       [0.00000000e+00, 1.42108547e-14, 8.88178420e-16, 2.66453526e-15],
       [5.55111512e-17, 0.00000000e+00, 0.00000000e+00, 4.44089210e-16],
       [8.88178420e-16, 6.21724894e-15, 0.00000000e+00, 1.42108547e-14]])

In [120]:
def lrange(*args, **kwargs):
    return list(range(*args, **kwargs))

irfobj = results.irf(var_decomp=(sigma_u / sd_u), periods=10)
orth_irfs = irfobj.orth_irfs

# cumulative impulse responses
irfs = (orth_irfs[:10] ** 2).cumsum(axis=0)

neqs = results.neqs
rng = lrange(neqs)
mse = results.mse(10)[:, rng, rng]

# lag x equation x component
fevd = np.empty_like(irfs)

for i in range(10):
    fevd[i] = (irfs[i].T / mse[i]).T

# switch to equation x lag x component
#decomp = fevd.swapaxes(0, 1)

In [152]:
steps = 10
ma_coefs = results.ma_rep(9)

k = len(results.sigma_u)
forc_covs = np.zeros((steps, k, k))

prior = np.zeros((k, k))
for h in range(steps):
    # Sigma(h) = Sigma(h-1) + Phi Sig_u Phi'
    phi = ma_coefs[h]
    var = phi @ results.sigma_u @ phi.T
    forc_covs[h] = prior = prior + var
forc_covs

array([[[0.69406084, 0.19866282, 0.07542932, 0.12525187],
        [0.19866282, 0.88773112, 0.06679637, 0.19003002],
        [0.07542932, 0.06679637, 1.42715106, 0.06422977],
        [0.12525187, 0.19003002, 0.06422977, 0.72200096]],

       [[0.71726523, 0.21086386, 0.07762947, 0.13556694],
        [0.21086386, 0.92522197, 0.09989765, 0.19786279],
        [0.07762947, 0.09989765, 1.48927624, 0.07490291],
        [0.13556694, 0.19786279, 0.07490291, 0.73020439]],

       [[0.78261253, 0.23370429, 0.07995612, 0.15431813],
        [0.23370429, 0.97427757, 0.12628964, 0.2116062 ],
        [0.07995612, 0.12628964, 1.5816593 , 0.09332389],
        [0.15431813, 0.2116062 , 0.09332389, 0.76725946]],

       [[0.85317743, 0.2698687 , 0.06926143, 0.17331434],
        [0.2698687 , 1.02196557, 0.13101231, 0.23299451],
        [0.06926143, 0.13101231, 1.69132926, 0.10093577],
        [0.17331434, 0.23299451, 0.10093577, 0.80573988]],

       [[0.94045189, 0.30923633, 0.07197224, 0.1972989 ],
      

In [154]:
ma_coefs

array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.18350846, -0.02424872, -0.00363372,  0.02199887],
        [ 0.06814117,  0.16377225,  0.05259282,  0.00431903],
        [-0.02275573,  0.09820135,  0.18333308,  0.04829864],
        [ 0.06467245, -0.01930672,  0.02517504,  0.06996402]],

       [[ 0.30767075, -0.00641004, -0.01285889,  0.00932753],
        [ 0.04825777,  0.2049666 ,  0.04305291,  0.01901118],
        [-0.01182419,  0.0205726 ,  0.24748445,  0.05634644],
        [ 0.05040629, -0.02434135,  0.01578916,  0.21592331]],

       [[ 0.30996154,  0.02198823, -0.022486  ,  0.01239301],
        [ 0.08674005,  0.19417692, -0.00985863,  0.02298701],
        [-0.05701899,  0.04514848,  0.27434849, -0.00235021],
        [ 0.02811824,  0.02500114,  0.00771659,  0.21441709]],



In [155]:
Phis

array([[[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.18350846,  0.06814117, -0.02275573,  0.06467245],
        [-0.02424872,  0.16377225,  0.09820135, -0.01930672],
        [-0.00363372,  0.05259282,  0.18333308,  0.02517504],
        [ 0.02199887,  0.00431903,  0.04829864,  0.06996402]],

       [[ 0.30767075,  0.04825777, -0.01182419,  0.05040629],
        [-0.00641004,  0.2049666 ,  0.0205726 , -0.02434135],
        [-0.01285889,  0.04305291,  0.24748445,  0.01578916],
        [ 0.00932753,  0.01901118,  0.05634644,  0.21592331]],

       [[ 0.30996154,  0.08674005, -0.05701899,  0.02811824],
        [ 0.02198823,  0.19417692,  0.04514848,  0.02500114],
        [-0.022486  , -0.00985863,  0.27434849,  0.00771659],
        [ 0.01239301,  0.02298701, -0.00235021,  0.21441709]],



In [145]:
results.mse(10)

array([[[0.69406084, 0.19866282, 0.07542932, 0.12525187],
        [0.19866282, 0.88773112, 0.06679637, 0.19003002],
        [0.07542932, 0.06679637, 1.42715106, 0.06422977],
        [0.12525187, 0.19003002, 0.06422977, 0.72200096]],

       [[0.71726523, 0.21086386, 0.07762947, 0.13556694],
        [0.21086386, 0.92522197, 0.09989765, 0.19786279],
        [0.07762947, 0.09989765, 1.48927624, 0.07490291],
        [0.13556694, 0.19786279, 0.07490291, 0.73020439]],

       [[0.78261253, 0.23370429, 0.07995612, 0.15431813],
        [0.23370429, 0.97427757, 0.12628964, 0.2116062 ],
        [0.07995612, 0.12628964, 1.5816593 , 0.09332389],
        [0.15431813, 0.2116062 , 0.09332389, 0.76725946]],

       [[0.85317743, 0.2698687 , 0.06926143, 0.17331434],
        [0.2698687 , 1.02196557, 0.13101231, 0.23299451],
        [0.06926143, 0.13101231, 1.69132926, 0.10093577],
        [0.17331434, 0.23299451, 0.10093577, 0.80573988]],

       [[0.94045189, 0.30923633, 0.07197224, 0.1972989 ],
      

In [146]:
denominator.cumsum(axis=0)

array([[[0.69406084, 0.19866282, 0.07542932, 0.12525187],
        [0.19866282, 0.88773112, 0.06679637, 0.19003002],
        [0.07542932, 0.06679637, 1.42715106, 0.06422977],
        [0.12525187, 0.19003002, 0.06422977, 0.72200096]],

       [[0.73390433, 0.21019034, 0.08078239, 0.13415639],
        [0.21019034, 0.92486466, 0.10191765, 0.1996849 ],
        [0.08078239, 0.10191765, 1.48022615, 0.08068433],
        [0.13415639, 0.1996849 , 0.08068433, 0.73037717]],

       [[0.8137135 , 0.23007019, 0.08717984, 0.15714781],
        [0.23007019, 0.9613197 , 0.119603  , 0.21042024],
        [0.08717984, 0.119603  , 1.57101013, 0.10930129],
        [0.15714781, 0.21042024, 0.10930129, 0.77287167]],

       [[0.90254594, 0.26189279, 0.06732058, 0.17878345],
        [0.26189279, 1.00363239, 0.13902531, 0.22821483],
        [0.06732058, 0.13902531, 1.67790352, 0.11255464],
        [0.17878345, 0.22821483, 0.11255464, 0.80922474]],

       [[0.99691438, 0.29995471, 0.05751131, 0.1900392 ],
      

$\mathrm{MSE}(h) = \sum_{i=0}^{h-1} \Phi \Sigma_u \Phi^T$

In [122]:
mse

array([[0.69406084, 0.88773112, 1.42715106, 0.72200096],
       [0.71726523, 0.92522197, 1.48927624, 0.73020439],
       [0.78261253, 0.97427757, 1.5816593 , 0.76725946],
       [0.85317743, 1.02196557, 1.69132926, 0.80573988],
       [0.94045189, 1.11821147, 1.83387395, 0.84210924],
       [0.98444108, 1.15520034, 1.90125637, 0.85656501],
       [1.03418221, 1.19070286, 1.96600415, 0.87223308],
       [1.07844559, 1.22009087, 2.02290186, 0.88457293],
       [1.11781577, 1.25087273, 2.0759561 , 0.89508382],
       [1.15067023, 1.273563  , 2.1170029 , 0.90308569]])

In [113]:
newde = np.zeros((10,4))
for i in range(10):
     newde[i] = np.diag(denominator[i])

In [114]:
newde.cumsum(0)

array([[0.69406084, 0.88773112, 1.42715106, 0.72200096],
       [0.73390433, 0.92486466, 1.48022615, 0.73037717],
       [0.8137135 , 0.9613197 , 1.57101013, 0.77287167],
       [0.90254594, 1.00363239, 1.67790352, 0.80922474],
       [0.99691438, 1.12097784, 1.80684022, 0.84472027],
       [1.05884017, 1.15975476, 1.86623974, 0.85804679],
       [1.12268523, 1.1968314 , 1.92452431, 0.87334247],
       [1.1817892 , 1.23284467, 1.97311774, 0.88364266],
       [1.2328996 , 1.27442603, 2.01680251, 0.89227024],
       [1.27886066, 1.30337859, 2.05020594, 0.89865438]])

In [ ]:
results.orth_ma_rep()

In [61]:
numerators.cumsum(axis=0)

array([[[0.69406084, 0.04445819, 0.00398667, 0.02172854],
        [0.05686377, 0.88773112, 0.00312634, 0.05001573],
        [0.00819753, 0.00502602, 1.42715106, 0.00571393],
        [0.02260325, 0.04067832, 0.0028907 , 0.72200096]],

       [[0.72531671, 0.05752855, 0.00405574, 0.03085299],
        [0.05748081, 0.91091438, 0.01847822, 0.0505951 ],
        [0.00909142, 0.00949646, 1.47691608, 0.00787395],
        [0.02377623, 0.04136701, 0.00687122, 0.72653139]],

       [[0.80057737, 0.07184923, 0.00417043, 0.04047302],
        [0.05922299, 0.9463773 , 0.01965633, 0.051259  ],
        [0.0096836 , 0.01292839, 1.56577829, 0.00946249],
        [0.02626311, 0.04591285, 0.01336346, 0.76391844]],

       [[0.87784968, 0.09396406, 0.00594993, 0.04764221],
        [0.0644747 , 0.98472746, 0.02421604, 0.05634608],
        [0.00970776, 0.01297705, 1.6721782 , 0.00993667],
        [0.02855078, 0.0504489 , 0.01347985, 0.79963248]],

       [[0.96504998, 0.10700517, 0.00786715, 0.05501127],
      

In [62]:
irfs

array([[[0.69406084, 0.04445819, 0.00398667, 0.02172854],
        [0.05686377, 0.88773112, 0.00312634, 0.05001573],
        [0.00819753, 0.00502602, 1.42715106, 0.00571393],
        [0.02260325, 0.04067832, 0.0028907 , 0.72200096]],

       [[0.72531671, 0.05752855, 0.00405574, 0.03085299],
        [0.05748081, 0.91091438, 0.01847822, 0.0505951 ],
        [0.00909142, 0.00949646, 1.47691608, 0.00787395],
        [0.02377623, 0.04136701, 0.00687122, 0.72653139]],

       [[0.80057737, 0.07184923, 0.00417043, 0.04047302],
        [0.05922299, 0.9463773 , 0.01965633, 0.051259  ],
        [0.0096836 , 0.01292839, 1.56577829, 0.00946249],
        [0.02626311, 0.04591285, 0.01336346, 0.76391844]],

       [[0.87784968, 0.09396406, 0.00594993, 0.04764221],
        [0.0644747 , 0.98472746, 0.02421604, 0.05634608],
        [0.00970776, 0.01297705, 1.6721782 , 0.00993667],
        [0.02855078, 0.0504489 , 0.01347985, 0.79963248]],

       [[0.96504998, 0.10700517, 0.00786715, 0.05501127],
      

In [51]:
CovUU / deviations - sigma_u / sd_u

array([[ 0.00000000e+00, -2.77555756e-17,  2.77555756e-17,
         5.55111512e-17],
       [ 0.00000000e+00,  1.11022302e-16,  2.77555756e-17,
        -8.32667268e-17],
       [ 1.38777878e-17,  2.77555756e-17,  0.00000000e+00,
         1.38777878e-17],
       [ 5.55111512e-17, -8.32667268e-17,  2.08166817e-17,
         1.11022302e-16]])

In [63]:
orth_irfs

array([[[ 0.83310314,  0.21085111,  0.06314009,  0.14740605],
        [ 0.23846126,  0.94219484,  0.05591365,  0.22364197],
        [ 0.09054019,  0.07089443,  1.19463428,  0.07559055],
        [ 0.15034377,  0.20168866,  0.05376522,  0.84970639]],

       [[ 0.1767933 ,  0.11432567, -0.00831089,  0.09552196],
        [ 0.02484018,  0.15226048,  0.12390271,  0.02406999],
        [ 0.02989801,  0.06686132,  0.22308075,  0.04647597],
        [ 0.03424887,  0.02624291,  0.06309135,  0.06730848]],

       [[ 0.27433678,  0.11966904,  0.01070915,  0.09808174],
        [ 0.04173945,  0.18831602,  0.03432372,  0.02576635],
        [ 0.02433476,  0.05858269,  0.29809765,  0.03985659],
        [ 0.04986857,  0.06742289,  0.08057447,  0.19335731]],

       [[ 0.27797896,  0.14871056, -0.0421841 ,  0.08467108],
        [ 0.07246866,  0.19583196,  0.06752559,  0.07132374],
        [ 0.00491565,  0.00697618,  0.32618999,  0.02177561],
        [ 0.04782965,  0.0673502 ,  0.01078833,  0.18898158]],



In [69]:
fe.sum(1)[:, None]

array([[1.16079718],
       [1.15850561],
       [1.02205437],
       [1.11178523]])

array([[1.16079718],
       [1.15850561],
       [1.02205437],
       [1.11178523]])

In [74]:
(Phis[h] @ CovUU) / deviations

array([[ 0.18948269,  0.11453269, -0.04682639,  0.07774798],
       [ 0.06239606,  0.12785781,  0.11381112,  0.06369662],
       [ 0.00820083,  0.04601356,  0.17774   ,  0.03548467],
       [ 0.02622768,  0.0293964 ,  0.0534173 ,  0.06033429]])

In [75]:
Phis[h] @ (CovUU / deviations)

array([[ 0.18948269,  0.11453269, -0.04682639,  0.07774798],
       [ 0.06239606,  0.12785781,  0.11381112,  0.06369662],
       [ 0.00820083,  0.04601356,  0.17774   ,  0.03548467],
       [ 0.02622768,  0.0293964 ,  0.0534173 ,  0.06033429]])